## Requirements

**Libraries**

In [ ]:
!pip install tensorboardX
!pip install --upgrade albumentations
!pip install --upgrade opencv-python

**Personal repository**

In [ ]:
!git clone https://github.com/Gianfranco-98/SSD_Tensorflow2.git
!mv -v SSD_Tensorflow2/* /content/
!rm -r SSD_Tensorflow2

## Dataset

In [3]:
import time

DATASET_NAME = 'VOC'
DATASET_KEY = '07+12'

start_dataset = time.time()

**COCO - Dataset preparing**

In [5]:
if DATASET_NAME == 'COCO':

    # Pycocotools
    !git clone https://github.com/cocodataset/cocoapi.git
    %cd cocoapi/PythonAPI
    !python setup.py install
    !make
    %cd ../..

    # Dataset
    !mkdir data
    %cd data
    !mkdir COCO2017
    %cd COCO2017
    !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip annotations_trainval2017.zip
    !rm annotations_trainval2017.zip
    %cd annotations
    !rm captions_train2017.json captions_val2017.json person_keypoints_train2017.json person_keypoints_val2017.json
    %cd /content

    print("Total dataset preparing time =", time.time() - start)

**VOC - Dataset preparing**

In [ ]:
if DATASET_NAME == 'VOC':
    !mkdir data
    %cd data
    !mkdir tmp
    %cd tmp

    # Downloading sub-datasets
    !wget pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
    !wget pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
    !wget pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
    if DATASET_KEY == '07++12':
        !wget pjreddie.com/media/files/VOC2012test.tar

    # Extract them in tmp folder 
    !echo "Extracting trainval2012..."
    !tar -xf VOCtrainval_11-May-2012.tar
    !mv -v VOCdevkit/VOC2012 /content/data/tmp/
    !rm -r VOCdevkit
    !echo "Extracting trainval2007..."
    !tar -xf VOCtrainval_06-Nov-2007.tar
    !mv -v VOCdevkit/VOC2007 /content/data/tmp/
    !rm -r VOCdevkit
    !echo "Extracting test2007..."
    !tar -xf VOCtest_06-Nov-2007.tar
    if DATASET_KEY == '07++12':
        !mv -v VOCdevkit/VOC2007 /content/data/tmp/
    elif DATASET_KEY == '07+12':
        !mv -v VOCdevkit/VOC2007 /content/data/tmp/VOC2007_Test
    if DATASET_KEY == '07++12':
        !echo "Extracting test2012..."
        !tar -xf VOC2012test.tar
        !mv -v VOCdevkit/VOC2012 /content/data/tmp/Test
    !rm -r VOCdevkit

    # First clean
    !rm VOCtrainval_11-May-2012.tar
    !rm VOCtrainval_06-Nov-2007.tar
    !rm VOCtest_06-Nov-2007.tar
    if DATASET_KEY == '07++12':
        !rm VOC2012test.tar
    %cd /content/data


    # Python script -------------------------------------------------------------- #
    import os
    from tqdm import tqdm
    import time
    import warnings

    DATASET_KEY_ = DATASET_KEY


    # Create VOC2012 train dir
    start = time.time()
    os.mkdir('VOC2012')
    os.mkdir('./VOC2012/JPEGImages')
    os.mkdir('./VOC2012/Annotations')
    if DATASET_KEY_ == '07++12':
        os.mkdir('./VOC2012/Test')
        os.mkdir('./VOC2012/Test/JPEGImages')
        os.mkdir('./VOC2012/Test/Annotations')
    elif DATASET_KEY_ == '07+12':
        os.mkdir('VOC2007')
        os.mkdir('./VOC2007/Test')
        os.mkdir('./VOC2007/Test/JPEGImages')
        os.mkdir('./VOC2007/Test/Annotations')
    else:
        raise ValueError("Datset key should be '07++12' or '07+12'")

    # VOC trainval2012
    os.chdir('./tmp/VOC2012/ImageSets/Main')
    trainval_names = open("trainval.txt").read().split('\n')[:-1]
    trainval_imgs_2012 = [name + '.jpg' for name in trainval_names]
    os.chdir('../../JPEGImages')
    print("\nAdding trainval2012 images...")
    for filename in tqdm(os.listdir()):
        if filename in trainval_imgs_2012:
            os.rename(filename, '../../../VOC2012/JPEGImages/' + filename)
    print("\nAdding trainval2012 annotations...")
    os.chdir('../Annotations')
    trainval_anns_2012 = []
    for filename in tqdm(os.listdir()):
        if filename[:-4] in trainval_names:
            trainval_anns_2012.append(filename)
            os.rename(filename, '../../../VOC2012/Annotations/' + filename)
    os.chdir('../../..')

    # VOC trainval2007
    os.chdir('./tmp/VOC2007/ImageSets/Main')
    trainval_names = open("trainval.txt").read().split('\n')[:-1]
    trainval_imgs_2007 = [name + '.jpg' for name in trainval_names]
    os.chdir('../../JPEGImages')
    print("\nAdding trianval2007 images...")
    for filename in tqdm(os.listdir()):
        if filename in trainval_imgs_2007:
            os.rename(filename, '../../../VOC2012/JPEGImages/' + filename)
    print("\nAdding trainval2007 annotations...")
    os.chdir('../Annotations')
    trainval_anns_2007 = []
    for filename in tqdm(os.listdir()):
        if filename[:-4] in trainval_names:
            trainval_anns_2007.append(filename)
            os.rename(filename, '../../../VOC2012/Annotations/' + filename)
    os.chdir('../../..')

    # VOC test 2007
    if DATASET_KEY_ == '07++12':
        imgs_2007_dst = '../../../VOC2012/JPEGImages/'
        anns_2007_dst = '../../../VOC2012/Annotations/'
    elif DATASET_KEY_ == '07+12':
        imgs_2007_dst = '../../../VOC2007/Test/JPEGImages/'
        anns_2007_dst = '../../../VOC2007/Test/Annotations/'
    else:
        # Insert alternative dataset configurations
        imgs_2007_dst, anns_2007_dst = None, None
    os.chdir('./tmp/VOC2007_Test/ImageSets/Main')
    test_names = open("test.txt").read().split('\n')[:-1]
    test_imgs_2007 = [name + '.jpg' for name in test_names]
    os.chdir('../../JPEGImages')
    print("\nAdding test2007 images...")
    for filename in tqdm(os.listdir()):
        if filename in test_imgs_2007:
            os.rename(filename, imgs_2007_dst + filename)
    print("\nAdding test2007 annotations...")
    os.chdir('../Annotations')
    test_anns_2007 = []
    for filename in tqdm(os.listdir()):
        if filename[:-4] in test_names:
            test_anns_2007.append(filename)
            os.rename(filename, anns_2007_dst + filename)
    os.chdir('../../..')

    # VOC test 2012
    if DATASET_KEY_ == '07++12':
        os.chdir('./tmp/Test/ImageSets/Main')
        test_names = open("test.txt").read().split('\n')[:-1]
        test_imgs_2012 = [name + '.jpg' for name in test_names]
        os.chdir('../../JPEGImages')
        print("\nAdding test2012 images...")
        for filename in tqdm(os.listdir()):
            if filename in test_imgs_2012:
                os.rename(filename, '../../../VOC2012/Test/JPEGImages/' + filename)
        warnings.warn("Missing Test Annotations in VOC2012")
        print("\nAdding test2012 annotations...")   
        os.chdir('../Annotations')
        test_anns_2012 = []
        for filename in tqdm(os.listdir()):
            if filename[:-4] in test_names:
                test_anns_2012.append(filename)
                os.rename(filename, '../../../VOC2012/Test/Annotations/' + filename)
        os.chdir('../../..')

    # Create train set
    if DATASET_KEY_ == '07++12':
        print("\nCreating Train set = VOC2012 trainval + VOC2007 trainval + VOC2007 test...")
        train_imgs = trainval_imgs_2012 + trainval_imgs_2007 + test_imgs_2007
        train_anns = trainval_anns_2012 + trainval_anns_2007 + test_anns_2007
    elif DATASET_KEY_ == '07+12':
        print("\nCreating Train set = VOC2012 trainval + VOC2007 trainval...")
        train_imgs = trainval_imgs_2012 + trainval_imgs_2007
        train_anns = trainval_anns_2012 + trainval_anns_2007
    else:
        # Insert alternative dataset configurations
        train_imgs, train_anns = None, None
    train_imgs.sort()
    train_anns.sort()

    # Create test set
    if DATASET_KEY_ == '07++12':
        print("Creating Test set = VOC2012 test...")
        test_imgs = test_imgs_2012
        test_anns = test_anns_2012
    elif DATASET_KEY_ == '07+12':
        print("Creating Test set = VOC2007 test...")
        test_imgs = test_imgs_2007
        test_anns = test_anns_2007
    else:
        # Insert alternative dataset configurations
        test_imgs, test_anns = None, None
    test_imgs.sort()
    test_anns.sort()

    print("Done in %f s" % (time.time() - start))
    # ---------------------------------------------------------------------------- #

    # Second clean
    !rm -r tmp
    %cd /content

    print("Total dataset preparing time =", time.time() - start_dataset)

##Hyperparameters

In [1]:
hyperparameters = {
    # Files parameters
    'DRIVE_PATH': '/content/drive/MyDrive',
    'CHECKPOINT_DIR': '/content/drive/MyDrive/Checkpoints/VOC/10000/DIoU',
    'CHECKPOINT_FILEPATH': '/content/drive/MyDrive/Checkpoints/VOC/10000/checkpoint',

    # Network parameters
    'BASE_WEIGHTS': 'imagenet',
    'BASE_NAME': "VGG16",
    'ASPECT_RATIOS': [[1., 2., 1/2],
                    [1., 2., 1/2, 3., 1/3],
                    [1., 2., 1/2, 3., 1/3],
                    [1., 2., 1/2, 3., 1/3],
                    [1., 2., 1/2],
                    [1., 2., 1/2]],
    'DEFAULT_BOXES_NUM': [4, 6, 6, 6, 4, 4],
    'INPUT_SHAPE': (300, 300, 3),
    'IMAGE_DIM': (300, 300),
    'N_CHANNELS': 3,

    # Learning parameters
    'WEIGHT_DECAY': 5e-4,
    'MOMENTUM': 0.9,
    'ALPHA': 1,
    'REGRESSION_TYPE': 'smooth_l1',

    # Train 
    'CHECKPOINT_PERIOD': 250,
    'PLOT_PERIOD': 500,
    'BATCH_SIZE': 32,
    'NUM_WORKERS': 8,
    'LOAD_MODEL': True,
    'TENSORBOARD_LOGS': False,

    # Data augmentation
    'IOU_THRESHOLDES': [0., 0.1, 0.3, 0.5, 0.7, 0.9],
    'PATCHFIND_ATTEMPTS': 50,

    # Inference
    'CONFIDENCE_THRESHOLD': 0.01,
    'JACCARD_THRESHOLD': 0.45,
    'MAX_NMS_BOXES': 200,
    'TOP_K_BOXES': 10,

    # Dataset configuration ------------------------------------------------------ #
    'DATASET_NAME': "VOC",
    'DATASET_YEAR': "2012",
    'TESTSET_YEAR': "2007",
    'DATASET_KEY': "07+12",
    'DATA_PATH': '/content/data/',
    'TRAINVAL_PATH': '/content/data/VOC2012',
    'TEST_PATH': '/content/data/VOC2007/Test',
    'TRAIN_ANN_PATH': None,
    'VAL_ANN_PATH': None,
    'ANN_PATH': None,

    # COCO Configuration
    #'TRAIN_ANN_PATH': '/content/data/COCO2017/annotations/instances_train/2017.json',
    #'VAL_ANN_PATH': '/content/data/COCO2017/annotations/instances_val/2017.json',
    #'TEST_ANN_PATH': '/content/data/COCO2017/annotations/instances_test/2017.json',
    #'SCALES': [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05],
    #'LR_VALUES': [1e-3, 1e-4, 1e-5, 1e-5],
    #'BOUNDARIES': [160000, 200000, 240000],
    # ------------------
    # VOC Configuration
    'IMGS_FOLDER': 'JPEGImages',
    'ANNS_FOLDER': 'Annotations',
    'TRAIN_ANN_PATH': '/content/data/VOC2012/Annotations',
    'VAL_ANN_PATH': '/content/data/VOC2007/Test/Annotations',
    'TEST_ANN_PATH': '/content/data/VOC2007/Test/Annotations',
    'SCALES': [0.10, 0.20, 0.37, 0.54, 0.71, 0.88, 1.05],
    'LR_VALUES': [1e-3, 1e-4, 1e-4],
    'BOUNDARIES': [60000, 80000],
    # ------------------
    'ITERATIONS': 80000
    # ---------------------------------------------------------------------------- #
}

In [2]:
keys = list(hyperparameters.keys())
with open('configuration.py', 'r') as readfile:
    lines = readfile.readlines()
with open('configuration.py', 'w') as writefile:
    for k in keys:
        if isinstance(hyperparameters[k], str):
            writefile.write(k + " = '" + hyperparameters[k] + "'")
        else:
            writefile.write(k + " = " + str(hyperparameters[k]))
        writefile.write("; ")

##Libraries

In [3]:
# Dataset
from pycocotools.coco import COCO

# Networks
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay

# Math
import numpy as np

# Generic
from tensorboardX import SummaryWriter
from google.colab import drive
from google.colab import files
import warnings
import time
import os

# My files
from ssd import SSD
from loss import SSD_Loss
from test import inference
from configuration import *
from detection_tools import *
from image_detection import *
from train_utilities import *
from data.dataset import COCO_Dataset, VOC_Dataset

## Initialization

**Dataset initialization**

In [ ]:
if DATASET_NAME == "COCO":
    train_coco = COCO(TRAIN_ANN_PATH)
    val_coco = COCO(VAL_ANN_PATH)
    test_coco = COCO(TEST_ANN_PATH)
    dataset = COCO_Dataset(
        train_coco,
        val_coco,
        test_coco
    )
elif DATASET_NAME == "VOC":
    train_roots = load_annotations(TRAIN_ANN_PATH)
    val_roots = load_annotations(VAL_ANN_PATH)
    test_roots = load_annotations(TEST_ANN_PATH)
    dataset = VOC_Dataset(
        train_roots,
        val_roots,
        test_roots
    )
else:
    raise ValueError("Wrong or unsupported dataset. Available 'COCO' or 'VOC'")
dataset.show_info()

In [5]:
dataloader = Dataloader(
    dataset, 
    BATCH_SIZE
)
train_generator = dataloader.generate_batch("train")

**Network initialization**

In [ ]:
ssd = SSD(num_classes=len(dataset.label_ids)+1, input_shape=INPUT_SHAPE)
checkpoint = tf.train.Checkpoint(ssd)
ssd.summary()                   

**Generate default Bounding Boxes**

In [7]:
fm_shapes = ssd.output_shape
aspect_ratios = ASPECT_RATIOS
scales = SCALES
default_boxes = Image.generate_default_boxes(fm_shapes, aspect_ratios, scales)

**Learning initializations**

In [8]:
learning_rate = PiecewiseConstantDecay(
    boundaries = BOUNDARIES,
    values = LR_VALUES
)
ssd_optimizer = SGD(
    learning_rate = learning_rate,
    momentum = MOMENTUM
)
ssd_loss = SSD_Loss(
    default_boxes = default_boxes,
    num_classes = ssd.num_classes, 
    regression_type = REGRESSION_TYPE, 
    hard_negative_ratio = 3, 
    alpha = ALPHA
)

**Tensorboard Writer**

In [9]:
if TENSORBOARD_LOGS:
    os.chdir('/content/')
    writer = SummaryWriter(comment = "SSD | __" + DATASET_NAME + DATASET_KEY + "__")

## Train

**Mount Drive and load last train data**

In [ ]:
# Training informations
last_iter = 0
iterations = []
mb_losses, loc_losses, conf_losses = [], [], []

# Mount to save checkpoints
drive.mount("/content/drive")
if LOAD_MODEL:
    print("Loading latest train data...")
    ssd, iterations, mb_losses, loc_losses, conf_losses = \
        load_train_data(ssd, CHECKPOINT_DIR)
    last_iter = iterations[-1]
    if TENSORBOARD_LOGS:
        for i in range(last_iter):
            writer.add_scalar("Multibox loss", mb_losses[i], i)
            writer.add_scalar("Confidence loss", conf_losses[i], i)
            writer.add_scalar("Localization loss", loc_losses[i], i)

**Train loop**

In [ ]:
for iteration in range(last_iter+1, ITERATIONS):

    # Load data
    #print("\n________Train iteration %d________" % iteration)
    #print("1.1 Data loading")
    glob_start = time.time()
    try:
        train_imgs, train_labels, train_ids = next(train_generator)
    except StopIteration:
        train_generator = dataloader.generate_batch("train")
        train_imgs, train_labels, train_ids = next(train_generator)
    batch_size = len(train_imgs)

    # Match bounding boxes
    #print(" - Matching bboxes...")
    matched_boxes, def_labels = [], []
    for b in range(batch_size):
        boxes, labels = match_boxes(train_labels[b], default_boxes)
        matched_boxes.append(boxes)
        def_labels.append(labels)

    # Predict and learn
    #print("2. Learning step")
    input_imgs = np.stack(train_imgs, axis=0)
    matched_boxes = tf.stack(matched_boxes, axis=0)
    def_labels = tf.stack(def_labels, axis=0)
    multibox_loss, localization_loss, confidence_loss = \
        learn(ssd, ssd_optimizer, ssd_loss, input_imgs, matched_boxes, def_labels)
    print("[%d] (%f s)   -   Multibox loss = |%f|, Localization loss = |%f|, Confidence_loss = |%f|" % 
          (iteration, time.time() - glob_start, multibox_loss, localization_loss, confidence_loss))
    
    # Plot train process
    iterations.append(iteration)
    mb_losses.append(multibox_loss)
    loc_losses.append(localization_loss)
    conf_losses.append(confidence_loss)
    if iteration % PLOT_PERIOD == 0 and iteration > 0:
        plot_train_data(iterations, mb_losses, loc_losses, conf_losses)

    # Update Tensorboard Writer
    if TENSORBOARD_LOGS:
        writer.add_scalar("Multibox loss", multibox_loss.numpy(), iteration)
        writer.add_scalar("Confidence loss", confidence_loss.numpy(), iteration)
        writer.add_scalar("Localization loss", localization_loss.numpy(), iteration)

    # Save checkpoint
    if iteration % CHECKPOINT_PERIOD == 0 and iteration > 0:
        print(" - Saving Train data...")
        save_train_data(checkpoint, CHECKPOINT_FILEPATH, iterations, mb_losses, loc_losses, conf_losses)

    #print("___Done in %f s!___" % (time.time() - glob_start))

## Test

In [ ]:
latest = tf.train.latest_checkpoint(CHECKPOINT_DIR)
ssd.load_weights(latest)

In [15]:
TEST_SIZE = 1
TEST_ITERATIONS = 10
dataloader = Dataloader(
    dataset, 
    TEST_SIZE
)
test_generator = dataloader.generate_batch("test")

In [ ]:
IMGS_ARRAY = []
for iteration in range(TEST_ITERATIONS):

    # Load data
    print("\n________Test iteration %d________" % iteration)
    print("1.1 Data loading")
    glob_start = time.time()
    try:
        test_imgs, test_labels, test_ids = next(test_generator)
    except StopIteration:
        test_generator = dataloader.generate_batch("test")
        test_imgs, test_labels, test_ids = next(test_generator)
    batch_size = len(test_imgs)
    IMGS_ARRAY.append(test_imgs)

    # Inference
    print("2. Inference")
    infer_time = time.time()
    input_imgs = np.stack(test_imgs, 0)
    vb, l, scores,  = inference(ssd, np.expand_dims(input_imgs[0], 0), default_boxes, 0.4)
    print("Inference time =", time.time() - infer_time)
    
    # Show ground truth image boxes
    gt_boxes = np.stack(test_labels[0], axis=0)[..., :-1]
    gt_labels = np.stack(test_labels[0], axis=0)[..., -1]
    test_bboxes(test_imgs[0], gt_boxes, 'min_max', gt_labels, dataset.classnames_dict)

    # Show predicted image boxes
    if vb.shape[0] != 0:
        test_bboxes(test_imgs[0], vb, 'min_max', l, dataset.classnames_dict, scores)
    else:
        print("No bboxes predicted")
    
    print("___Done in %f s!___" % (time.time() - glob_start))

**Compute mAP**

*Credits*: https://github.com/Cartucho/mAP

In [ ]:
# 1. Clone repository
!git clone https://github.com/Cartucho/mAP
for f in tqdm(os.listdir('./mAP/input/ground-truth')):
    filename = './mAP/input/ground-truth/' + f
    !rm $filename
for f in tqdm(os.listdir('./mAP/input/detection-results')):
    filename = './mAP/input/detection-results/' + f
    !rm $filename

# 2. Convert labels
for i in tqdm(range(len(dataset.test_ids))):

    # Load data
    try:
        test_imgs, test_labels, test_ids = next(test_generator)
    except StopIteration:
        test_generator = dataloader.generate_batch("test")
        test_imgs, test_labels, test_ids = next(test_generator)

    # Inference
    input_imgs = np.stack(test_imgs, 0)
    vb, l, scores = inference(ssd, np.expand_dims(input_imgs[0], 0), default_boxes, 0.3)
    if vb.shape[0] > 0:
        if np.max(vb) <= 1.5:
            vb = np.clip(vb, 0, 1)
            vb[..., 0] = vb[..., 0] * input_imgs[0].shape[1]
            vb[..., 1] = vb[..., 1] * input_imgs[0].shape[0]
            vb[..., 2] = vb[..., 2] * input_imgs[0].shape[1]
            vb[..., 3] = vb[..., 3] * input_imgs[0].shape[0]
        vb = np.array(vb, dtype=np.int32)
      
    # Get ground truth image boxes
    gt_boxes = np.stack(test_labels[0], axis=0)[..., :-1]
    gt_labels = np.stack(test_labels[0], axis=0)[..., -1]
    if np.max(gt_boxes) <= 1.5:
        gt_boxes = np.clip(gt_boxes, 0, 1)
        gt_boxes[..., 0] = gt_boxes[..., 0] * input_imgs[0].shape[1]
        gt_boxes[..., 1] = gt_boxes[..., 1] * input_imgs[0].shape[0]
        gt_boxes[..., 2] = gt_boxes[..., 2] * input_imgs[0].shape[1]
        gt_boxes[..., 3] = gt_boxes[..., 3] * input_imgs[0].shape[0]
    gt_boxes = np.array(gt_boxes, dtype=np.int32)        

    # Create the files
    name = test_ids[0] + '.txt'
    ## 1. GT
    with open(name, 'w') as writefile:
        lines = []
        for j in range(len(gt_labels)):
            b = gt_boxes[j]
            char = "" if j == 0 else "\n"
            lines.append(char + dataset.classnames_dict[gt_labels[j]] + " " + str(b[0]) + " " + str(b[1]) + " " + str(b[2]) + " " + str(b[3]))
        writefile.writelines(lines)
    os.rename(name, './mAP/input/ground-truth/' + name)
    ## 2. Pred
    with open(name, 'w') as writefile:
        if vb.shape[0] > 0:
            lines = []
            for j in range(len(l)):
                b = vb[j]
                char = "" if j == 0 else "\n"
                lines.append(char + dataset.classnames_dict[l[j]] + " " + str(scores[j]) + " " + str(b[0]) + " " + str(b[1]) + " " + str(b[2]) + " " + str(b[3]))
            writefile.writelines(lines)
    os.rename(name, './mAP/input/detection-results/' + name)

# 3. Compute mAP
%cd /content/mAP/
!python main.py -na

**Test with specific images**

In [ ]:
imgs = os.listdir('/content/data/VOC2007/Test/JPEGImages')[0:32]
for i in range(len(imgs)):
    imgs[i] = cv2.resize(cv2.imread(imgs[i]), (300, 300))
img = imgs[10]
plt.figure(figsize=(8, 8))
plt.imshow(img)
plt.show()

In [ ]:
vb, l, scores = inference(
                          ssd, 
                          np.expand_dims(preprocess_input(img), 0), 
                          default_boxes, 
                          conf_threshold=0.3,
                          loc_threshold=0.45,
                          num_nms_output=250,
                          top_k=10
)

# Show predicted image boxes
test_bboxes(img, vb, l, dataset.classnames_dict, scores)